In [1]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle as cPickle  # Sorry, workaround for Python 3.4
import gzip

# Third-party libraries
import numpy as np

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding='latin1')  # Make it work with Py3.x
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [2]:
%time training_data, validation_data, test_data = load_data_wrapper()

CPU times: user 1.65 s, sys: 493 ms, total: 2.15 s
Wall time: 2.15 s


In [3]:
%time x, y = zip(*training_data)

x = np.array(x)
x = x.reshape(50000, 784)

y = np.array(y)
y = y.reshape(50000, 10)

CPU times: user 48.3 ms, sys: 27.7 ms, total: 75.9 ms
Wall time: 76.4 ms


In [4]:
import keras


import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils

from sklearn.preprocessing import StandardScaler

np.random.seed(1337) # for reproducibility


def preprocess_data(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler



Using Theano backend.


/Users/tjl/Development/sandpit/lca16/source_packages/Theano/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [5]:
 
nb_classes = y.shape[1]
print(nb_classes, 'classes')

dims = x.shape[1]
print(dims, 'dims')

 


10 classes
784 dims


In [6]:

print("Building model...")

LAYER_ONE_SIZE = 1024

model = Sequential()
model.add(Dense(output_dim=LAYER_ONE_SIZE, input_dim=dims, init='glorot_uniform'))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(output_dim=LAYER_ONE_SIZE, input_dim=LAYER_ONE_SIZE, init='glorot_uniform'))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(output_dim=LAYER_ONE_SIZE, input_dim=LAYER_ONE_SIZE, init='glorot_uniform'))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(output_dim=nb_classes, input_dim=LAYER_ONE_SIZE, init='glorot_uniform'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam")

print("Training model...")

%time model.fit(x, y, nb_epoch=8, batch_size=128, validation_split=0.15)  

Building model...
Training model...
Train on 42500 samples, validate on 7500 samples
Epoch 1/8
42500/42500 [==============================] - 31s - loss: 0.6849 - val_loss: 0.3958
Epoch 2/8
42500/42500 [==============================] - 33s - loss: 0.2618 - val_loss: 0.2013
Epoch 3/8
42500/42500 [==============================] - 28s - loss: 0.1844 - val_loss: 0.2019
Epoch 4/8
42500/42500 [==============================] - 29s - loss: 0.1455 - val_loss: 0.1508
Epoch 5/8
42500/42500 [==============================] - 29s - loss: 0.1270 - val_loss: 0.1434
Epoch 6/8
42500/42500 [==============================] - 28s - loss: 0.1080 - val_loss: 0.1399
Epoch 7/8
42500/42500 [==============================] - 31s - loss: 0.0959 - val_loss: 0.1442
Epoch 8/8
42500/42500 [==============================] - 27s - loss: 0.0849 - val_loss: 0.1255
CPU times: user 6min 52s, sys: 28.3 s, total: 7min 21s
Wall time: 4min


In [7]:
p_x, p_y = zip(*test_data)

p_x = np.array(p_x)
p_x = p_x.reshape(10000, 784)

p_y = np.array(p_y)
p_y = p_y.reshape(10000)

In [8]:
%time preds = model.predict(p_x)
print(preds[1])
print(np.argmax(preds[1]))

CPU times: user 1.81 s, sys: 25.7 ms, total: 1.83 s
Wall time: 649 ms
[  2.50913968e-06   3.34689976e-03   9.96577501e-01   3.75878626e-05
   1.43854220e-06   2.39443523e-07   2.07008270e-06   2.75669845e-05
   4.16515286e-06   4.00953004e-09]
2


In [9]:
pred_idx = [np.argmax(a) for a in preds]

In [10]:
pairs = zip(pred_idx, p_y)
print(len(p_y))

10000


In [11]:
number_correct = sum([int(a == b) for a, b in pairs])
print(number_correct)

9732


In [12]:
number_incorrect = len(p_y) - number_correct
print(number_incorrect)

268


In [13]:
eps = 0.000001  # avoid division by zero
success_rate = number_correct / float((number_correct + number_incorrect + eps))
print(success_rate)

0.9731999999026799


In [14]:
4 !=2

True